In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from projects.feed.rank.src import config
from projects.feed.rank.src.model import Model
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
product='newmse'
mark='tuwen'

In [4]:
# 载入模型
model_hour = '2020050819'
valid_hour = '2020050822'
# valid_hour = '2020030812'
model_root = f'/search/odin/publicData/CloudS/libowei/rank_online/data/{mark}_hour_sgsapp_v1/models/8'
# model_root = '/home/gezi/tmp/rank/exps/tuwen/v29/dlrm-norm/'
model_dir = f'{model_root}/{model_hour}'
# model_dir = model_root
tfrecord_root = f'/search/odin/publicData/CloudS/libowei/rank/{product}/data/{mark}_hour_{product}_v1/tfrecords'
tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
print('model:', model_dir)
print('record:', tfrecord_dir)
argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
FLAGS(argv)
FLAGS.eager = True
# FLAGS.fields_norm=True
config.init()
model = Model()
melt.eager.restore(model, model_dir)

model: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020050819
record: /search/odin/publicData/CloudS/libowei/rank/newmse/data/tuwen_hour_newmse_v1/tfrecords/2020050822


2020-05-11 19:20:00 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-05-11 19:20:00 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-05-11 19:20:00 0:00:00 batch_size: 512 eval_batch_size: 512 batch_size_per_gpu: 512 num_gpus: 0 gpu: [] CUDA_VISIABLE_DEVICES=[-1] work_mode: train distributed: False horovod: False
2020-05-11 19:20:00 0:00:00 model: [8] model_dir: [/search/odin/libowei/rank/tuwen/model/8]
2020-05-11 19:20:00 0:00:00 Latest checkpoint: /search/odin/publicData/CloudS/libowei/rank_online/data/tuwen_hour_sgsapp_v1/models/8/2020050819/model.ckpt-2020050819-1.00-9677


In [5]:
dataset = Dataset('valid')

In [6]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

4162592

In [7]:
batch_size = 512
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [8]:
def get_info(x, mid):
  mids = gezi.decode(x['mid'].numpy())
  hits = []
  for i in range(len(mids)):
    if mids[i] == mid:
      hits += [i]
  res = [{}] * len(hits)
  for key in x:
    for i, pos in enumerate(hits):
      res[i][key] = tf.convert_to_tensor([x[key][pos]])
  return res

In [9]:
xs = []
for i, (x, _) in tqdm(enumerate(batches), total=num_steps, ascii=True, desc='loop'):
  xs += get_info(x, '00000000-56ef-e6bf-ffff-ffffba81c941')

loop: 100%|##########| 8131/8131 [01:38<00:00, 82.16it/s] 


In [10]:
def get_doc(xs, docid):
  for x in xs:
    if tf.compat.as_str_any(x['docid'][0].numpy()) == docid:
      return x

In [11]:
# 这两个doc 第一个位于位置0 未点击           它被称为“黄金木”，价值比黄花梨还高，却是农民最讨厌的树
# 第二个位于位置8 点击  时长30              阴历几月出生的人，外有志气，内有才华，十之八九能成为人中龙凤
x1 = get_doc(xs, 'open_doc_prod28249148')
x2 = get_doc(xs, 'open_doc_prod28226024')

In [12]:
model(x1)
x_all1 = model.deep.x_all

In [13]:
model(x2)
x_all2 = model.deep.x_all

In [14]:
def rename(field):
  for x in maps:
#     print(x, len(x))
#     print('0', field, x[0], x[1])
    field = field.replace(x[0], x[1])
#     print('1', field, x[0], x[1])
  return field

In [15]:
other_fields = [x.name.split('/')[-2] for x in model.variables if x.name.endswith('embeddings:0')and not '/emb/emb/' in x.name]
other_fields[model.deep.history_index:model.deep.history_index] = model.deep.history_fields
onehot_fields = gezi.get_global('fields')
maps = [x.strip().split() for x in open('../conf/fields_map.txt')]
onehot_fields2 = [rename(x) for x in onehot_fields]
fields = other_fields + onehot_fields2
fields2 = other_fields + onehot_fields
# [(x, y) for x, y in zip(onehot_fields, onehot_fields2)]

In [18]:
import copy
x_all2_npy = x_all2.numpy()
x_all1_npy = x_all1.numpy()

In [19]:
l = []
for i in tqdm(range(len(fields))):
  x_all = x_all1_npy.copy()
  x_all[0][i] = x_all2_npy[0][i]
  x1['x_all'] = x_all
  model(x1)
  prob_click1 = model.prob_click.numpy()[0][0]
  x_all = x_all2_npy.copy()
  x_all[0][i] = x_all1_npy[0][i]
  x2['x_all'] = x_all
  model(x2)
  prob_click2 = model.prob_click.numpy()[0][0]
  l.append((i, fields[i], fields2[i], prob_click1, prob_click2, prob_click2 - prob_click1))
  l.sort(key=lambda x: x[-1], reverse=True)
gezi.pprint(pd.DataFrame(l, columns=['fid', 'field', 'ori_field', 'pclick1', 'pclick2', 'delta']))

100%|██████████| 94/94 [00:03<00:00, 28.30it/s]


 +-------+------------------------------+-----------------+-----------+-----------+------------+
|   fid | field                        | ori_field       |   pclick1 |   pclick2 |      delta |
|-------+------------------------------+-----------------+-----------+-----------+------------|
|    29 | IArtKw2                      | IATKWSE         |  0.169473 | 0.15809   | -0.0113828 |
|    26 | IAccountId                   | IACID           |  0.179297 | 0.141027  | -0.0382702 |
|     1 | doc_emb                      | doc_emb         |  0.216182 | 0.11776   | -0.0984218 |
|    72 | MInterestArtTopic            | MITATTP         |  0.232059 | 0.101405  | -0.130654  |
|    71 | MInterestArtTag              | MITATTG         |  0.237353 | 0.10318   | -0.134173  |
|    64 | MInterestArtPornScoreKw      | MITATPORNSKW    |  0.238853 | 0.100525  | -0.138328  |
|    42 | ICFRecallWord                | ICFRW           |  0.248397 | 0.1076    | -0.140797  |
|    66 | MInterestArtPornScoreTopic  